In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [2]:
# Remove the double quotes around the path string
# Correct the path as needed
anime_df=pd.read_csv(r'/content/drive/MyDrive/Machine Learning Projects/anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
rating_df=pd.read_csv(r'/content/drive/MyDrive/Machine Learning Projects/rating.csv')
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [4]:
rating_df=rating_df[rating_df.user_id<=10000]

In [5]:
rating_df['rating']=rating_df['rating'].replace(-1,pd.NA)

In [6]:
rating_df=rating_df.drop_duplicates(['user_id','anime_id'])

In [7]:
user_item_matrix=rating_df.pivot(index='user_id',columns='anime_id',values='rating')

In [8]:
user_similarity=cosine_similarity(user_item_matrix.fillna(0))

<ipython-input-8-5ef536a01b45>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_similarity=cosine_similarity(user_item_matrix.fillna(0))


In [9]:
def user_based_recommendations(user_id,user_item_matrix,user_similarity,n=5):
  user_scores=user_similarity[user_id - 1 ]
  similar_users=sorted(enumerate(user_scores),key=lambda x:x[1],reverse=True)
  recommended_anime=[]
  for user,score in similar_users[1:]:
    rated_anime = user_item_matrix.loc[user_item_matrix.index[user - 1]]
    positively_rated_anime = rated_anime[rated_anime >= 6].index
    target_user_rated_anime = user_item_matrix.loc[user_id - 1].dropna().index
    recommended_anime.extend(set(positively_rated_anime) - set(target_user_rated_anime))
    if len(recommended_anime) >= n:
      break
  return recommended_anime[:n]

In [10]:
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)

<ipython-input-10-5098c16ba4a5>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)


In [11]:
def item_based_recommendations(user_id, user_item_matrix, item_similarity, n=5):
    positively_rated_anime = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] >= 6].index
    recommended_anime = []
    for anime_id in positively_rated_anime:
        similar_anime = item_similarity[anime_id - 1]
        similar_anime_sorted = sorted(enumerate(similar_anime), key=lambda x: x[1], reverse=True)

        for anime, score in similar_anime_sorted[1:]:  # Exclude the liked anime
            user_rated_anime = user_item_matrix.loc[user_id].dropna().index
            if anime not in user_rated_anime:
                recommended_anime.append(anime)
            if len(recommended_anime) >= n:
                break
    return recommended_anime[:n]

In [12]:
user_id=9
user_recommendations=user_based_recommendations(user_id,user_item_matrix,user_similarity,n=5)
item_recommendations=item_based_recommendations(user_id,user_item_matrix,user_similarity,n=5)

In [20]:

print(f"User-based recommendations for user {user_id}: {user_recommendations}")
for anime_id in user_recommendations:
  # Check if the anime_id exists in anime_df before trying to access
  anime_info = anime_df.loc[anime_df['anime_id']==anime_id]
  if not anime_info.empty:
    anime_name=anime_info['name'].values[0]
    print(f"Anime ID: {anime_id}, Anime Name: {anime_name}")
  else:
    print(f"Anime ID: {anime_id}, Anime Name: Not Found in anime_df")


print(f"Item-based recommendations for user {user_id}: {item_recommendations}")
for anime_id in item_recommendations:
  # Check if the anime_id exists in anime_df before trying to access
  anime_info = anime_df.loc[anime_df['anime_id']==anime_id]
  if not anime_info.empty:
    anime_name=anime_info['name'].values[0]
    print(f"Anime ID: {anime_id}, Anime Name: {anime_name}")
  else:
    print(f"Anime ID: {anime_id}, Anime Name: Not Found in anime_df")

User-based recommendations for user 9: [12293, 28677, 31240, 11785, 25099]
Anime ID: 12293, Anime Name: Campione!: Matsurowanu Kamigami to Kamigoroshi no Maou
Anime ID: 28677, Anime Name: Yamada-kun to 7-nin no Majo (TV)
Anime ID: 31240, Anime Name: Re:Zero kara Hajimeru Isekai Seikatsu
Anime ID: 11785, Anime Name: Haiyore! Nyaruko-san
Anime ID: 25099, Anime Name: Ore ga Ojousama Gakkou ni &quot;Shomin Sample&quot; Toshite Gets♥Sareta Ken
Item-based recommendations for user 9: [9081, 9778, 9947, 1979, 794]
Anime ID: 9081, Anime Name: Not Found in anime_df
Anime ID: 9778, Anime Name: Toriko Hime: Hakudaku Mamire no Reijou
Anime ID: 9947, Anime Name: Lan Mao
Anime ID: 1979, Anime Name: Tobira wo Akete (1995)
Anime ID: 794, Anime Name: Not Found in anime_df
